### Note: remember running "mamba activate caiman" in the terminal before using this code

# CaImAn Pipeline
This notebook demonstrates how Caiman uses different functions to denoise, deconvolve and demix neurons from our calcium imaging data.
This particular notebook is a modified version of Caiman's "demo_pipeline", so you can reference it for more detailed descriptions of steps and/or functions.

In [1]:
import bokeh.plotting as bpl
import cv2
import glob
import logging
import matplotlib.pyplot as plt
import numpy as np
import os

try:
    cv2.setNumThreads(0)
except():
    pass

try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes
        # when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
bpl.output_notebook()

/tmp/ipykernel_532293/1449603744.py:18: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('load_ext autoreload')
/tmp/ipykernel_532293/1449603744.py:19: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('autoreload 2')
2023-11-15 18:23:25.253229: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-15 18:23:25.427665: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX51

Loading BokehJS ...

### Select file(s) to be processed
#### Remember to change "base_dir"

In [2]:
base_dir = "/mnt/data/erika-organoid-data/CODE/CURRENT-FOLDER/LOOP/exp1-2_11-25_07_23/" # Define the folder path where your data is saved in

original_dir = os.getcwd()  # Remember the original directory so you can revert back after processing

all_files = {}

for organoid in range(1, 10):  # Lops through organoid1 through organoid9 folders
    organoid_folder = os.path.join(base_dir, f"organoid{organoid}")
    
    if not os.path.exists(organoid_folder):
        continue

    for run in range(8):  # For run through run_7
        run_folder = os.path.join(organoid_folder, f"run_{run}" if run > 0 else "run")
        
        if not os.path.exists(run_folder):
            continue

        # Skip this folder if analysis_results.hdf5 already exists inside it
        if os.path.exists(os.path.join(run_folder, 'analysis_results.hdf5')):
            print(f"Skipping organoid{organoid} run{run} as analysis_results.hdf5 already exists.")
            continue
        
        fnames = []
        base_file_name = os.path.join(run_folder, f"run{run}.tif")
        
        if os.path.exists(base_file_name):
            fnames.append(os.path.basename(base_file_name))  # Only keep the filename, not the full path

      # Loop to find any files that match the pattern run#_X#.tif
        for x in range(1, 6):  # Assuming max number of files is 5, i.e., run#_X1 to run#_X5
            extra_file_name = os.path.join(run_folder, f"run{run}_X{x}.tif")
            if os.path.exists(extra_file_name):
                fnames.append(os.path.basename(extra_file_name))  # Only keep the filename, not the full path

        # End of the loop that populates fnames
        print(f"For organoid{organoid} run{run}, files found: {fnames}")

        all_files[f"organoid{organoid}_run{run}"] = fnames

        # Change the current working directory to the run folder
        os.chdir(run_folder)


        
        # ---- Start of processing code ----


        # Set up logger (optional):
        logging.basicConfig(format=
                    "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.WARNING)



        # Setup some parameters:

        # dataset dependent parameters
        fr = 20                             # imaging rate in frames per second
        decay_time = 0.4                    # length of a typical transient in seconds

        # motion correction parameters
        strides = (48, 48)          # start a new patch for pw-rigid motion correction every x pixels
        overlaps = (24, 24)         # overlap between pathes (size of patch strides+overlaps)
        max_shifts = (6,6)          # maximum allowed rigid shifts (in pixels)
        max_deviation_rigid = 3     # maximum shifts deviation allowed for patch with respect to rigid shifts
        pw_rigid = True             # flag for performing non-rigid motion correction

        # parameters for source extraction and deconvolution
        p = 1                       # order of the autoregressive system (erika comment: use p=1 for when you are using a quick calcium transient, e.g. the spike occurs within a frame. Use p=2 when you have slow rise time)
        gnb = 2                     # number of global background components (erika comment: also 1 or 2 for same reason as above)
        merge_thr = 0.85            # merging threshold, max correlation allowed (erika comment: usually somewhere between 0.8 and 0.9, sometimes even 0.95)
        rf = 15                     # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
        stride_cnmf = 6             # amount of overlap between the patches in pixels
        K = 4                       # number of components per patch (erika comment: basically tries to look for K number of neurons per patch (video said it was okay to overshoot a bit because at the end you "clean up" components anyway")
        gSig = [4, 4]               # expected half size of neurons in pixels
        method_init = 'greedy_roi'  # initialization method (if analyzing dendritic data using 'sparse_nmf')
        ssub = 1                    # spatial subsampling during initialization
        tsub = 1                    # temporal subsampling during intialization

        # parameters for component evaluation
        min_SNR = 2.0               # signal to noise ratio for accepting a component (erika comment: in general thhis is 2 or 3)
        rval_thr = 0.85             # space correlation threshold for accepting a component
        cnn_thr = 0.99              # threshold for CNN based classifier
        cnn_lowest = 0.1            # neurons with cnn probability lower than this value are rejected


        
        
        # Create a parameters object
        opts_dict = {'fnames': fnames,
                    'fr': fr,
                    'decay_time': decay_time,
                    'strides': strides,
                    'overlaps': overlaps,
                    'max_shifts': max_shifts,
                    'max_deviation_rigid': max_deviation_rigid,
                    'pw_rigid': pw_rigid,
                    'p': p,
                    'nb': gnb,
                    'rf': rf,
                    'K': K, 
                    'gSig': gSig,
                    'stride': stride_cnmf,
                    'method_init': method_init,
                    'rolling_sum': True,
                    'only_init': True,
                    'ssub': ssub,
                    'tsub': tsub,
                    'merge_thr': merge_thr, 
                    'min_SNR': min_SNR,
                    'rval_thr': rval_thr,
                    'use_cnn': True,
                    'min_cnn_thr': cnn_thr,
                    'cnn_lowest': cnn_lowest}
        opts = params.CNMFParams(params_dict=opts_dict)

        # Start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
        if 'dview' in locals():
            cm.stop_server(dview=dview)
        c, dview, n_processes = cm.cluster.setup_cluster(
            backend='local', n_processes=None, single_thread=False)

        # First we create a motion correction object with the parameters specified
        mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))

        # Run piecewise-rigid motion correction using NoRMCorre
        mc.motion_correct(save_movie=True)
        m_els = cm.load(mc.fname_tot_els)
        border_to_0 = 0 if mc.border_nan == 'copy' else mc.border_to_0 
        # maximum shift to be used for trimming against NaNs

        # MEMMORY MAPPING the file in order 'C'
        fname_new = cm.save_memmap(mc.mmap_file, base_name='memmap_', order='C',
                           border_to_0=border_to_0, dview=dview) # exclude borders

        # Now load the file
        Yr, dims, T = cm.load_memmap(fname_new)
        images = np.reshape(Yr.T, [T] + list(dims), order='F') 
        # load frames in python format (T x X x Y)

        # Restart cluster to clean up memory
        cm.stop_server(dview=dview)
        c, dview, n_processes = cm.cluster.setup_cluster(
        backend='local', n_processes=None, single_thread=False)

        # RUN CNMF ON PATCHES
            # First extract spatial and temporal components on patches and combine them
            # for this step deconvolution is turned off (p=0). If you want to have
            # deconvolution within each patch change params.patch['p_patch'] to nonzero value
        cnm = cnmf.CNMF(n_processes, params=opts, dview=dview)
        cnm = cnm.fit(images)

        # Plot contours of found components (optional for looping)
        # Cn = cm.local_correlations(images.transpose(1,2,0))
        # Cn[np.isnan(Cn)] = 0
        # cnm.estimates.plot_contours_nb(img=Cn)

        #%% RE-RUN seeded CNMF on accepted patches to refine and perform deconvolution 
        cnm2 = cnm.refit(images, dview=dview)

        # COMPONENT EVALUATION
        # the components are evaluated in three ways:
        #   a) the shape of each component must be correlated with the data
        #   b) a minimum peak SNR is required over the length of a transient
        #   c) each shape passes a CNN based classifier
        cnm2.estimates.evaluate_components(images, cnm2.params, dview=dview)

        # Extract DF/F values  	(Computes DF/F normalized fluorescence for the extracted traces)
        cnm2.estimates.detrend_df_f(quantileMin=8, frames_window=250)

        # Select only high quality components
        cnm2.estimates.select_components(use_object=True)

        # DECONVOLUTION 
        cnm2.estimates.deconvolve(cnm2.params, dview=None, dff_flag=True)
    
        # Display final resutls (optional for looping) - shows cell masks and corresponding signal
        # cnm2.estimates.nb_view_components(img=Cn, denoised_color='red')
        # print('you may need to change the data rate to generate this one: use jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10 before opening jupyter notebook')

        # You can save an hdf5 file with all the fields of the cnmf object
        save_results = True
        if save_results:
            cnm2.save('analysis_results.hdf5')

        #%% STOP CLUSTER and clean up log files
        cm.stop_server(dview=dview)
        log_files = glob.glob('*_LOG_*')
        for log_file in log_files:
            os.remove(log_file)


        
        # ---- End of processing code ----
        

        
        # After processing, change back to the original directory
        os.chdir(original_dir)


For organoid3 run3, files found: ['run3.tif', 'run3_X1.tif', 'run3_X2.tif']


  0%|                                                    | 0/3 [00:00<?, ?it/s]

Decode mmap filename run3_els__d1_540_d2_640_d3_1_order_F_frames_3086.mmap


 33%|██████████████▋                             | 1/3 [01:18<02:37, 78.51s/it]

Decode mmap filename run3_X1_els__d1_540_d2_640_d3_1_order_F_frames_3087.mmap


...

Decode mmap filename /run/media/silviu/mega1/mega/Erika/LOOP/organoid3/run_3/memmap__d1_540_d2_640_d3_1_order_C_frames_9260.mmap
Decode mmap filename /run/media/silviu/mega1/mega/Erika/LOOP/organoid3/run_3/memmap__d1_540_d2_640_d3_1_order_C_frames_9260.mmap


     6815712 [components_evaluation.py:classify_components_ep():243] [543773] Component 49 is only active jointly with neighboring components. Space correlation calculation might be unreliable.


Decode mmap filename /run/media/silviu/mega1/mega/Erika/LOOP/organoid3/run_3/memmap__d1_540_d2_640_d3_1_order_C_frames_9260.mmapDecode mmap filename /run/media/silviu/mega1/mega/Erika/LOOP/organoid3/run_3/memmap__d1_540_d2_640_d3_1_order_C_frames_9260.mmap

Decode mmap filename /run/media/silviu/mega1/mega/Erika/LOOP/organoid3/run_3/memmap__d1_540_d2_640_d3_1_order_C_frames_9260.mmap
Decode mmap filename /run/media/silviu/mega1/mega/Erika/LOOP/organoid3/run_3/memmap__d1_540_d2_640_d3_1_order_C_frames_9260.mmap
Decode mmap filename /run/media/silviu/mega1/mega/Erika/LOOP/organoid3/run_3/memmap__d1_540_d2_640_d3_1_order_C_frames_9260.mmap
GPU run not requested, disabling use of GPUs
USING MODEL (keras API): /home/silviu/caiman_data/model/cnn_model.json


2023-11-15 20:19:17.973626: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-11-15 20:19:17.973686: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: ggwserv1
2023-11-15 20:19:17.973691: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: ggwserv1
2023-11-15 20:19:17.974235: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 545.23.6
2023-11-15 20:19:17.974444: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 545.23.6
2023-11-15 20:19:17.974448: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 545.23.6


69/69 [==============================] - 3s 28ms/step
